In [0]:
import tensorflow as tf
import numpy as np
from google.colab import files # for file upload

tf.set_random_seed(777)  # for reproducibility

In [4]:
#파일업로드창 출력
uploaded = files.upload()

#업로드한 파일정보 출력
for fn in uploaded.keys():
  print('Upload file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving data-04-zoo.csv to data-04-zoo.csv
Upload file "data-04-zoo.csv" with length 4485 bytes


In [7]:
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1] #x축 전부, y축은 마지막 제외하고 전부
y_data = xy[:, [-1]] #x축 전부, Y축은 마지막만

print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [8]:

nb_classes = 7  # 0 ~ 6  # softmax에 의해 구분될 class가 0~6까지 총 7개

X = tf.placeholder(tf.float32, [None, 16])  # x축은 N개, Y축은 16개
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6  #X축은 N개 Y축은 1개

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot : 0~6의 값을 0or1로 표시하기 위함, 그런데 shape=(?, 1, 7) array가 2개가 아닌3개가 됨->reshape필
print("one_hot:", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # Y_one_hot 전체를 nb_classes로 매핑되도록 reshape : shape=(?, 7)
print("reshape one_hot:", Y_one_hot)


one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [9]:

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

Instructions for updating:
Colocations handled automatically by placer.


In [0]:

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)


In [0]:
# softmax_cross_entropy_with_logits_v2로 간단하게 적용: logits, labels를 설정
# Cross entropy cost/loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)


In [0]:

prediction = tf.argmax(hypothesis, 1) # softmax분류에서 최대값 1개를 찾음
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #label값과 예측한 값을 비교
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #비교한 값을 평균하여 정확도 계산


In [14]:

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #트레이
    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # 최종 도출된 모델의 가설(prediction<-hypothesis)에서 테스트해봄(테스트data와 같아서 문제긴함)
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


Step:     0	Cost: 5.480	Acc: 37.62%
Step:   100	Cost: 0.806	Acc: 79.21%
Step:   200	Cost: 0.488	Acc: 88.12%
Step:   300	Cost: 0.350	Acc: 90.10%
Step:   400	Cost: 0.272	Acc: 94.06%
Step:   500	Cost: 0.222	Acc: 95.05%
Step:   600	Cost: 0.187	Acc: 97.03%
Step:   700	Cost: 0.161	Acc: 97.03%
Step:   800	Cost: 0.141	Acc: 97.03%
Step:   900	Cost: 0.124	Acc: 97.03%
Step:  1000	Cost: 0.111	Acc: 97.03%
Step:  1100	Cost: 0.101	Acc: 99.01%
Step:  1200	Cost: 0.092	Acc: 100.00%
Step:  1300	Cost: 0.084	Acc: 100.00%
Step:  1400	Cost: 0.078	Acc: 100.00%
Step:  1500	Cost: 0.072	Acc: 100.00%
Step:  1600	Cost: 0.068	Acc: 100.00%
Step:  1700	Cost: 0.064	Acc: 100.00%
Step:  1800	Cost: 0.060	Acc: 100.00%
Step:  1900	Cost: 0.057	Acc: 100.00%
Step:  2000	Cost: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: